# 네이버 영화평 감성분석 
- TfidVectorizer

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


In [2]:
train_df = pd.read_csv('data/naver_movie_train.tsv',sep='\t')
test_df = pd.read_csv('data/naver_movie_test.tsv',sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


- Tokenizer 함수 정의

In [3]:
from konlpy.tag import Okt
okt = Okt()


In [4]:
stop_text = '은 는 이 가 을 를 의 에게 에 들 좀 잘 과 도 으로 하는 자 와 ㅋㅋ ㅎㅎ ㅠㅠ'
stopwords = set(stop_text.split())


In [5]:
def okt_tokenizer(text):
    morphs = okt.morphs(text, stem=True)
    tokens = [word for word in morphs if word not in stopwords]
    return tokens

- TfidfVectorizer

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer(
    tokenizer=okt_tokenizer,ngram_range=(1,2), max_df=0.9
)

In [7]:
%time tvect.fit(train_df.document)

Wall time: 3min 9s


TfidfVectorizer(max_df=0.9, ngram_range=(1, 2),
                tokenizer=<function okt_tokenizer at 0x0000021CDF8F1310>)

In [10]:
%time X_train_tv = tvect.transform(train_df.document)

Wall time: 4min 4s


In [11]:
%time X_test_tv = tvect.transform(test_df.document)

Wall time: 1min 17s


In [12]:
y_train = train_df.label.values
y_test = test_df.label.values

- Naive Bayes 분류기

In [14]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.get_params()

{'alpha': 1.0, 'class_prior': None, 'fit_prior': True}

In [15]:
nb.fit(X_train_tv,y_train)
nb.score(X_test_tv,y_test)

0.8605558460362788

In [21]:
reviews = ['두시간 반이 왜 이렇게 짧냐','저 정말 스파이더맨 팬입니다. 정말 역겨움을 참느라 힘들었습니다. 이 영화가 재미있다고 하는 사람들 이해할 수가 없네요. ']

In [23]:
reviews_tv = tvect.transform(reviews)
nb.predict(reviews_tv)

array([0, 0], dtype=int64)

In [24]:
nb.predict

<bound method _BaseNB.predict of MultinomialNB()>